# Streamlit で PDF の Webアプリを作る

In [ ]:
import os
import PyPDF2
import glob
import base64
import streamlit as st
import shutil

In [ ]:
if __name__ == "__main__":
    filename = './data/sample.pdf'
    splitpath = './split_out/'
    os.makedirs(splitpath, exist_ok=True)
    split(filename, splitpath)
    mergepath = './merge_out/'
    os.makedirs(splitpath, exist_ok=True)
    merge(splitpath, mergepath)

In [ ]:
def split(filename, outpath, selected_item, zipname):
    reader = PyPDF2.PdfFileReader(filename)
    page_n = reader.getNumPages()
    for page in range(page_n):
        split_page = PyPDF2.PdfFileWriter()
        split_page.addPage(reader.getPage(page))
        savepath = outpath + str(page) + '.pdf'
        with open(savepath, 'wb') as f:
            split_page.write(f)
        if selected_item == 'pdf':
            with open(savepath, 'rb') as f:
                b64 = base64.b64encode(f.read()).decode()
            href = f'<a href="data:application/octet-stream;base64,{b64}" download="str(page) + .pdf">download:{page}.pdf</a>'
            st.markdown(href, unsafe_allow_html=True)
    if selected_item == 'zip':
        shutil.make_archive(zipname, 'zip', outpath)
        with open('tmp_zip'+'.zip', 'rb') as f:
            b64 = base64.b64encode(f.read()).decode()
            href = f'<a href="data:application/octet-stream;base64,{b64}" download="pdf-controller.zip">download:zip</a>'
            st.markdown(href, unsafe_allow_html=True)

In [ ]:
def merge(pdfdir, outdir):
    merger = PyPDF2.PdfFileMerger()
    for filepath in glob.glob(pdfdir + '*.pdf'):
        merger.append(filepath)
    merger.write(outdir + 'merge.pdf')
    merger.close()

# streamlt なし ver

In [ ]:
import os
import PyPDF2
import glob

def merge(pdfdir, outdir):
    merger = PyPDF2.PdfFileMerger()
    for filepath in glob.glob(pdfdir + '*.pdf'):
        merger.append(filepath)
    merger.write(outdir + 'merge.pdf')
    merger.close()
    
def split(filename, outpath, selected_item, zipname):
    reader = PyPDF2.PdfFileReader(filename)
    page_n = reader.getNumPages()
    for page in range(page_n):
        split_page = PyPDF2.PdfFileWriter()
        split_page.addPage(reader.getPage(page))
        savepath = outpath + str(page) + '.pdf'
        with open(savepath, 'wb') as f:
            split_page.write(f)
            
if __name__ == "__main__":
    filename = './data/sample.pdf'
    splitpath = './split_out/'
    os.makedirs(splitpath, exist_ok=True)
    split(filename, splitpath)
    mergepath = './merge_out/'
    os.makedirs(splitpath, exist_ok=True)
    merge(splitpath, mergepath)

# 書籍

In [ ]:
# 複数のファイルを一つにまとめる
from PyPDF2 import PdfFileMerger
import os

mgr = PdfFileMerger()
filelist = os.listdir("pdfs")
filelist = [f for f in filelist if '.pdf' in f]
print(filelist)

for filename in filelist:
    with open(os.path.join("pdfs", filename), 'rb') as input:
        mgr.append(input)
        
output = open(os.path.join("pdfs", "pdf_all.pdf"), "wb")
mgr.write(output)
output.close()

In [ ]:
# 1ページごとに分割する
from PyPDF2 import PdfFileReader
from PyPDF2 import PdfFileWriter

fp = open("pdf_all.pdf", 'rb')
input = PdfFileReader(fp)

pgnum = input.getNumPages()

for i in range(pgnum):
    output = PdfFileWriter()
    output.addPage(input.getPage(i))
    outputfile = open("pdf_all_"+str(i)+".pdf", 'wb')
    output.write(outputfile)
    outputfile.close()

fp.close()

# Streamlit で自作

In [ ]:
#仕様
# 一回ばらして、選択したものだけ結合できる
# 一枚ずつほしい時は、zipにする
# Folderを渡されたら、Folder内のpdfを結合する
#